In [24]:
import heapq
import os
import sys

class myPriorQueue:
  def __init__(self):
    self.arr = []

  def size(self):
    return len(self.arr)

  def push(self, item):
    return heapq.heappush(self.arr, item)

  def pop(self):
    return heapq.heappop(self.arr)

  def has(self, item):
    return item in self.arr

  def get_priority(self, val):
    for x in self.arr:
      if x[1] == val:
        return x[0]
    return None

  def replace(self, val, priority):
    for i in range(len(self.arr)):
      if self.arr[i][1] == val:
        self.arr[i][0] = priority
        heapify(self.arr)
        return True
    return False
    


class Maze:
  def __init__(self, data):
    self.edge = data[0][0]
    self.size = self.edge**2
    self.start = 0
    self.goal = data[-1][0]
    self.graph = data[1:-1]

  def get_path(self, node, paths):
    res = [node]
    while paths[node] != -1:
      node = paths[node]
      res.append(node)
    return res[::-1]

  def Heuristic_Manhattan(self, node):
    rows = abs((self.goal % self.edge) - (node % self.edge))
    cols = abs(int(self.goal / self.edge) - int(node / self.edge))
    return rows + cols

  def BFS(self):
    frontier = [self.start]
    explored_nodes = []
    paths = {self.start: -1}

    while len(frontier) > 0:
      current_node = frontier.pop(0)
      explored_nodes.append(current_node)
      neighbors = self.graph[current_node]
      for x in neighbors:
        if x not in explored_nodes and x not in frontier:
          paths[x] = current_node
          if x == self.goal:
            explored_nodes.append(x)
            return True, len(explored_nodes), explored_nodes, self.get_path(x, paths)
          frontier.append(x)
    return False, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)
  
  def UCS(self):
    explored_nodes = []
    paths = {self.start: -1}
    frontier = myPriorQueue()
    frontier.push((0, self.start))
    while frontier.size() > 0:
      current_cost, current_node = frontier.pop()
      explored_nodes.append(current_node)
      if current_node == self.goal:
        return True, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)
      neighbors = self.graph[current_node]
      for x in neighbors:
        x_not_in_frontier = frontier.get_priority(x) is None
        x_cost = current_cost + 1
        if not x in explored_nodes and x_not_in_frontier:
          frontier.push((x_cost, x))
          paths[x] = current_node
        elif not x_not_in_frontier and x_cost < frontier.get_priority(x):
          frontier.replace(x, x_cost)
    return False, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)
  
  def DLS(self, start, explored_nodes, paths, depth):
    copy_paths = paths[:]
    if depth == 0 or start in paths:
      return False, explored_nodes, copy_paths
    explored_nodes.append(start)
    copy_paths.append(start)
    if start == self.goal:
      return True, explored_nodes, copy_paths
    neighbors = self.graph[start]
    for x in neighbors:
      res = self.DLS(x, explored_nodes, copy_paths, depth - 1)
      if res[0]:
        return res
    return False, explored_nodes, copy_paths
  
  def IDS(self):
    found = False
    explored_nodes = []
    paths = []
    times = 0
    for i in range(self.size):
      found, explored, paths = self.DLS(self.start, [], [], i+1)
      explored_nodes.append(explored)
      times += len(explored)
      if found:
        return True, times, explored_nodes, paths
    return False, times, explored_nodes, paths

  def GBFS(self):
    explored_nodes = []
    paths = {self.start: -1}
    frontier = myPriorQueue()
    frontier.push((self.Heuristic_Manhattan(self.start), self.start))
    while frontier.size() > 0:
      current_cost, current_node = frontier.pop()
      explored_nodes.append(current_node)
      if current_node == self.goal:
        return True, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)
      neighbors = self.graph[current_node]
      for x in neighbors:
        x_not_in_frontier = frontier.get_priority(x) is None
        x_heuristic = self.Heuristic_Manhattan(x)
        if not x in explored_nodes and x_not_in_frontier:
          frontier.push((x_heuristic, x))
          paths[x] = current_node
        elif not x_not_in_frontier and x_heuristic < frontier.get_priority(x):
          frontier.replace(x, x_heuristic)
    return False, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)

  def A_Star(self):
    explored_nodes = []
    paths = {self.start: -1}
    frontier = myPriorQueue()
    frontier.push((self.Heuristic_Manhattan(self.start), self.start))
    while frontier.size() > 0:
      current_cost, current_node = frontier.pop()
      explored_nodes.append(current_node)
      if current_node == self.goal:
        return True, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)
      neighbors = self.graph[current_node]
      for x in neighbors:
        x_not_in_frontier = frontier.get_priority(x) is None
        x_heuristic = self.Heuristic_Manhattan(x) + len(self.get_path(current_node, paths))
        if not x in explored_nodes and x_not_in_frontier:
          frontier.push((x_heuristic, x))
          paths[x] = current_node
        elif not x_not_in_frontier and x_heuristic < frontier.get_priority(x):
          frontier.replace(x, x_heuristic)
    return False, len(explored_nodes), explored_nodes, self.get_path(current_node, paths)




if __name__ == '__main__':
  which_data = input("Pick data set (1, 2 or 3): ")

  fi = open(os.path.join(sys.path[0], "input{}.txt".format(which_data)), "r")
  lines = fi.readlines()
  fi.close()

  data = []
  for line in lines:
    tmp = []
    for x in line.strip().split():
      tmp.append(int(x))
    data.append(tmp)

  print ("\nInput is: {}\n\nOutput is:".format(data))
  data = [sorted(x) for x in data]

  maze = Maze(data)
  
  fo = open(os.path.join(sys.path[0], "output{}.txt".format(which_data)), "w")
  searchs_name = ["1. Breadth-first search", "2. Uniform-cost search", "3. Iterative deepening search", "4. Greedy-best first search", "5. Tree-search A*"]
  results = [maze.BFS(), maze.UCS(), maze.IDS(), maze.GBFS(), maze.A_Star()]

  for i in range(len(searchs_name)):
    name = searchs_name[i] + ": (Start node: {}; Goal node: {})\n>>> Result:\n".format(maze.start, maze.goal)
    fo.write(name)
    print (name[:-1])
    if results[i][0]:
      time = " - The time to escape the maze: {} minutes\n".format(results[i][1])
      if type(results[i][2][0]) == int:
        explored = " - The list of explored nodes: {}\n".format(results[i][2])
      else:
        explored = " - The list of explored nodes:\n"
        for j in range(len(results[i][2])):
          explored += "    * Depth = {}: {}\n".format(j, results[i][2][j])
      paths = " - The list of nodes on the paths found: {}\n\n".format(results[i][3])
      fo.write(" - Found the paths to goal\n")
      print (" - Found the paths to goal")
      fo.write(time)
      print (time[:-1])
      fo.write(explored)
      print (explored[:-1])
      fo.write(paths)
      print (paths)
    else:
      fo.write(" - The paths to goal not found")
      print (" - The paths to goal not found")
      time = " - The time to escape the maze: {} minutes\n".format(results[i][1])
      if type(results[i][2][0]) == int:
        explored = " - The list of explored nodes: {}\n".format(results[i][2])
      else:
        explored = " - The list of explored nodes:\n"
        for j in range(len(results[i][2])):
          explored += "    * Depth = {}: {}\n".format(j, results[i][2][j])
      print (explored)
      fo.write(explored + "\n")

  fo.close()

  print (">>> The result is also printed to file output.txt in the same folder <<<")
